In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import csc_matrix

In [2]:
data_dir = '../data/processed/'

In [3]:
rating_df = pd.read_csv(data_dir + 'train_rating.csv')
rating_df.shape

(12800168, 5)

In [4]:
mean_ratings = rating_df.groupby('movieId').rating.mean().reset_index()
mean_ratings.head()

,movieId,rating
0,1,3.920535
1,2,3.209256
2,3,3.146716
3,4,2.856856
4,5,3.070523


In [5]:
movies_df = pd.read_csv(data_dir + 'movie.csv')
# movies_df = movies_df[movies_df.movieId.isin(mean_ratings.movieId)]
movies_df.head()

,movieId,title,action,adventure,animation,children,comedy,crime,documentary,drama,...,imax,musical,mystery,romance,scifi,thriller,unknown,war,western,year
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995.0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995.0
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1995.0
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,1995.0
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995.0


In [16]:
movies_df[['movieId','title']].sample(10)

,movieId,title
20288,99746,"Samaritan, The (2012)"
1543,1595,Free Willy 3: The Rescue (1997)
22821,109173,Storm Rider (2013)
6217,6316,"Wiz, The (1978)"
23534,112056,Backstairs (Hintertreppe) (1921)
19606,97035,"Living Skeleton, The (Kyûketsu dokuro-sen) (1968)"
16173,81758,Faces of Schlock (2005)
6211,6310,Memories of Me (1988)
19349,96131,K-20: The Fiend with Twenty Faces (2008)
26649,128171,Beethoven's 5th (2003)


In [38]:
cross = movies_df.merge(mean_ratings, how='inner', on='movieId')
cross.head()

,movieId,title,action,adventure,animation,children,comedy,crime,documentary,drama,...,musical,mystery,romance,scifi,thriller,unknown,war,western,year,rating
0,1,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1995.0,3.920535
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1995.0,3.209256
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1995.0,3.146716
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,1995.0,2.856856
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1995.0,3.070523


In [39]:
null_year = cross[cross.year.isna()].index
null_year.shape

(15,)

In [40]:
cross.drop(index=null_year, inplace=True)

In [41]:
cross.drop(columns='title', inplace=True)

In [42]:
from sklearn.preprocessing import MinMaxScaler

In [43]:
scaler = MinMaxScaler()
scaler.fit(cross)
features = scaler.transform(cross)

In [44]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(
    # Quantidade de recomendações
    n_neighbors=6,
    # Algoritmo
    algorithm='brute',
    # metrica de distancia
    metric='minkowski'
)

model.fit(features)

NearestNeighbors(algorithm='brute', n_neighbors=6)

In [50]:
sample = cross[cross.movieId == 1]
sample = scaler.transform(sample)

In [51]:
distances, indexes = model.kneighbors(sample)
indexes = indexes[0]
indexes

array([    0,  3027,  4789,  3921,  2209, 10063])

In [52]:
recs = cross.loc[indexes].merge(movies_df[['movieId','title']], on='movieId', how='inner')

In [53]:
recs

,movieId,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,...,mystery,romance,scifi,thriller,unknown,war,western,year,rating,title
0,1,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1995.0,3.920535,Toy Story (1995)
1,3114,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1999.0,3.843933,Toy Story 2 (1999)
2,4886,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,2001.0,3.878958,"Monsters, Inc. (2001)"
3,4016,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,2000.0,3.588933,"Emperor's New Groove, The (2000)"
4,2294,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1998.0,3.302023,Antz (1998)
5,33463,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1990.0,3.363636,DuckTales: The Movie - Treasure of the Lost La...
